<a href="https://colab.research.google.com/github/Kunal-1669/deep_learning_Final_Project/blob/main/Untitled17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Preprocessing Dataset

In [1]:
#load the dataset
from datasets import load_dataset
data=load_dataset('lavita/ChatDoctor-HealthCareMagic-100k')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
df=pd.DataFrame(data['train'])

In [3]:
df.shape


(112165, 3)

In [4]:
df = df.drop('instruction', axis=1)

In [5]:
df

,input,output
0,I woke up this morning feeling the whole room ...,"Hi, Thank you for posting your query. The most..."
1,My baby has been pooing 5-6 times a day for a ...,Hi... Thank you for consulting in Chat Doctor....
2,"Hello, My husband is taking Oxycodone due to a...","Hello, and I hope I can help you today.First, ..."
3,lump under left nipple and stomach pain (male)...,HI. You have two different problems. The lump ...
4,I have a 5 month old baby who is very congeste...,Thank you for using Chat Doctor. I would sugge...
...,...,...
112160,im 25 years old ..i started using mtp kit on 5...,"Hello, Thanks for letting us know your health ..."
112161,My 5 year old son has been coughing for a mont...,As you have mentioned in your history that you...
112162,My toes on right foot more than left are numb ...,Hi. The numbness and blue discoloration could ...
112163,"I was diagnosis with pleurisy last Tuesday, an...",Thanks for your question on Chat Doctor. Treat...


Preprocessing

In [6]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    return ' '.join(lemmatized_tokens)


In [7]:
import re

def text_strip(text):
    cleaned_text = re.sub("[\t\r\n]+", ' ', str(text)).lower()  # remove escape characters
    cleaned_text = re.sub("(__+|-+|~+|\++)", ' ', cleaned_text).lower()  # remove repeating _, -, ~, or + characters
    cleaned_text = re.sub("(\.\.+)", ' ', cleaned_text).lower()  # remove repeating . characters
    cleaned_text = re.sub("\s+", ' ', cleaned_text).lower()  # remove multiple spaces
    cleaned_text = re.sub("\s.\s", ' ', cleaned_text).lower()  # remove any single characters hanging between 2 spaces
    cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text).lower() # remove punctuation
    cleaned_text = re.sub(r'http\S+', '', cleaned_text).lower() # remove URL
    cleaned_text = re.sub(r'[{}()\[\]]', '', cleaned_text).lower()  # remove all types of braces

    return cleaned_text



In [8]:
# Apply the cleaning function to 'input' and 'output' columns
df['input'] = df['input'].apply(text_strip)
df['output'] = df['output'].apply(text_strip)
df['input'] = df['input'].apply(preprocess_text)
df['output'] = df['output'].apply(preprocess_text)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import matplotlib.pyplot as plt

2023-12-10 15:30:40.184240: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>


False

In [11]:
df[['input', 'output']] = df[['input', 'output']].apply(lambda x: x.apply(lambda y: word_tokenize(y.lower())))


In [12]:
nltk.download('stopwords')

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>


False

In [13]:
df

,input,output
0,"[woke, morning, feeling, whole, room, spinning...","[hi, thank, posting, query, likely, cause, sym..."
1,"[baby, pooing, 6, time, day, week, last, day, ...","[hi, thank, consulting, chat, doctor, seems, k..."
2,"[hello, husband, taking, oxycodone, due, broke...","[hello, hope, help, todayfirst, medication, ta..."
3,"[lump, left, nipple, stomach, pain, male, hii,...","[hi, two, different, problem, lump, nipple, re..."
4,"[5, month, old, baby, congested, terrible, cou...","[thank, using, chat, doctor, would, suggest, s..."
...,...,...
112160,"[im, 25, year, old, started, using, mtp, kit, ...","[hello, thanks, letting, u, know, health, conc..."
112161,"[year, old, son, coughing, month, taken, docto...","[mentioned, history, son, coughing, since, mon..."
112162,"[toe, right, foot, left, numb, turning, blue, ...","[hi, numbness, blue, discoloration, could, due..."
112163,"[diagnosis, pleurisy, last, tuesday, still, sh...","[thanks, question, chat, doctor, treatment, pl..."


In [14]:
def remove_stop_words(tokens):
    stop_words_set = set(stopwords.words('english'))
    return [word for word in tokens if word.lower() not in stop_words_set]


In [15]:
from nltk.corpus import stopwords

In [16]:
from nltk.corpus import stopwords
df['input'] = df['input'].apply(remove_stop_words)
df['output'] = df['output'].apply(remove_stop_words)


print(df)

                                                    input  \
0       [woke, morning, feeling, whole, room, spinning...   
1       [baby, pooing, 6, time, day, week, last, day, ...   
2       [hello, husband, taking, oxycodone, due, broke...   
3       [lump, left, nipple, stomach, pain, male, hii,...   
4       [5, month, old, baby, congested, terrible, cou...   
...                                                   ...   
112160  [im, 25, year, old, started, using, mtp, kit, ...   
112161  [year, old, son, coughing, month, taken, docto...   
112162  [toe, right, foot, left, numb, turning, blue, ...   
112163  [diagnosis, pleurisy, last, tuesday, still, sh...   
112164  [within, past, hour, husband, developed, rash,...   

                                                   output  
0       [hi, thank, posting, query, likely, cause, sym...  
1       [hi, thank, consulting, chat, doctor, seems, k...  
2       [hello, hope, help, todayfirst, medication, ta...  
3       [hi, two, different

In [17]:
from nltk.stem import PorterStemmer

def stem_tokenized_text(tokenized_text):
    # Initialize the PorterStemmer
    stemmer = PorterStemmer()

    # Perform stemming on each token
    stemmed_tokens = [stemmer.stem(token) for token in tokenized_text]

    return stemmed_tokens

In [18]:
df['input'] = df['input'].apply(stem_tokenized_text)
df['output'] = df['output'].apply(stem_tokenized_text)

Word Embedding

In [20]:
import spacy

# Load spaCy model
nlp = spacy.load('en_core_web_sm')



In [21]:
# Function to apply word embeddings using spaCy
def get_word_embeddings(word_list):
    text = ' '.join(word_list)
    doc = nlp(text)
    return doc.vector

In [22]:
df['WordEmbeddings_input'] = df['input'].apply(get_word_embeddings)

In [ ]:
df['WordEmbeddings_output'] = df['output'].apply(get_word_embeddings)

In [ ]:
csv_path = '/Users/vivianwang/Desktop/data.csv' 
df.to_csv(csv_path, index=False)


In [ ]:
df.to_csv('/Users/vivianwang/Desktop/data.csv', index=False) 

In [ ]:
import numpy as np

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import matplotlib.pyplot as plt

In [ ]:
csv_path = "/Users/vivianwang/Desktop/data2.csv"
df = pd.read_csv(csv_path)
df

,input,output,WordEmbeddings_input,WordEmbeddings_output
0,"['woke', 'morn', 'feel', 'whole', 'room', 'spi...","['hi', 'thank', 'post', 'queri', 'like', 'caus...",[-0.0619953 -0.6736265 0.27994376 -0.085758...,[-0.18782409 -0.88191456 0.14123142 0.173059...
1,"['babi', 'poo', 'time', 'day', 'week', 'last',...","['hi', 'thank', 'consult', 'chat', 'doctor', '...",[ 0.31923333 -0.6726571 -0.16014402 -0.239708...,[-0.11867172 -0.927781 0.11605667 0.053519...
2,"['hello', 'husband', 'take', 'oxycodon', 'due'...","['hello', 'hope', 'help', 'todayfirst', 'medic...",[ 0.07557318 -0.8664425 0.2868909 0.074551...,[-0.07790519 -0.9422826 0.19836313 0.008373...
3,"['lump', 'left', 'nippl', 'stomach', 'pain', '...","['hi', 'two', 'differ', 'problem', 'lump', 'ni...",[ 1.45712122e-01 -7.75916994e-01 1.59821466e-...,[-0.0350829 -0.5795279 0.00846534 0.203456...
4,"['month', 'old', 'babi', 'congest', 'terribl',...","['thank', 'use', 'chat', 'doctor', 'would', 's...",[-0.00440924 -0.8600714 0.1022485 0.140155...,[-0.11640272 -0.8963055 0.17284362 0.040940...
...,...,...,...,...
112160,"['im', '25', 'year', 'old', 'start', 'use', 'm...","['hello', 'thank', 'let', 'u', 'know', 'health...",[-0.0474515 -0.73382026 -0.0372104 0.085389...,[-0.01673762 -1.057229 0.26074028 0.190503...
112161,"['year', 'old', 'son', 'cough', 'month', 'take...","['mention', 'histori', 'son', 'cough', 'sinc',...",[ 0.11159492 -0.9231173 0.00670093 -0.010550...,[-0.14276338 -0.991815 0.02161139 0.257161...
112162,"['toe', 'right', 'foot', 'left', 'numb', 'turn...","['hi', 'numb', 'blue', 'discolor', 'could', 'd...",[ 0.00540123 -0.5803814 0.22330971 0.105037...,[-0.05790729 -1.046325 0.32568908 0.108866...
112163,"['diagnosi', 'pleurisi', 'last', 'tuesday', 's...","['thank', 'question', 'chat', 'doctor', 'treat...",[-1.41046569e-01 -7.67617047e-01 7.36642182e-...,[-0.13445875 -0.8021505 0.33348566 0.051080...
